# Create CloneMaps Sophie BEP

In [1]:
import shutil
import subprocess
from glob import glob
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import yaml
import pandas as pd

from cartopy import crs
from cartopy import feature as cfeature
from cartopy.io import shapereader

import esmvaltool

## Settings for all catchments and periods

In [2]:
FLUXDIR = Path('/lustre1/0/wtrcycle/users/jaerts/camels_wflow/sophie_BEP/')
MODELDIR = Path('/lustre1/0/wtrcycle/comparison/pcrglobwb2_input/')
OUTPUTDIR = Path('/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/global_05min/cloneMaps/SophieBEP/')
flux_selection_file = 'fluxnet_toren_selectie_klein_Sophie_Jaren.csv'

## Create DataFrame containing gridcell bounding box

In [3]:
# Load FLUXNET selection DataFrame
df = pd.read_csv(f'{FLUXDIR}/{flux_selection_file}', sep=';')
df = df.set_index('SITE_ID')

In [4]:
# Get station lon lat
longitude = df['LOCATION_LONG'][0]
latitude = df['LOCATION_LAT'][0]

In [5]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def get_pixel_cell_coords(longitude, latitude, grid_resolution):
    lons = np.arange(-180, 180, grid_resolution)
    lats = np.arange(-90, 90, grid_resolution)
    
    lon_min = find_nearest(lons, longitude)
    lon_max = lon_min + grid_resolution
    
    lat_min = find_nearest(lats, latitude)
    lat_max = lat_min + grid_resolution
    
    return (lon_min, lon_max, lat_min, lat_max)

def add_buffer(lonmin, lonmax, latmin, latmax, buffer=float):
    if lonmin > 0:
        lonmin = lonmin - buffer
    if lonmin < 0:
        lonmin = lonmin + buffer
    if lonmax > 0:
        lonmax = lonmax + buffer
    if lonmax < 0:
        lonmax = lonmax - buffer

    if latmin > 0:
        latmin = latmin - buffer
    if latmin < 0:
        latmin = latmin + buffer
    if latmax > 0:
        latmax = latmax + buffer
    if latmax < 0:
        latmax = latmax - buffer
            
    return (lonmin, lonmax, latmin, latmax)
    

## Construct new Dataframe

In [6]:
# ['Site_ID', 'Lon_min', 'lon_max', 'lat_min', 'lat_max']
sites = []
lons_min = []
lons_max = []
lats_min = []
lats_max = []

for site_id, row in df.iterrows():
    sites.append(site_id)
    longitude = row.LOCATION_LONG
    latitude = row.LOCATION_LAT
    
    coords = get_pixel_cell_coords(longitude, latitude, 0.5)
    
    lons_min.append(coords[0])
    lons_max.append(coords[1])
    lats_min.append(coords[2])
    lats_max.append(coords[3])
    
df_sites = pd.DataFrame()
df_sites['site_id'] = sites
df_sites['lon_min'] = lons_min
df_sites['lon_max'] = lons_max
df_sites['lat_min'] = lats_min
df_sites['lat_max'] = lats_max
df_sites = df_sites.set_index('site_id')
df_sites.to_csv(f'{FLUXDIR}/clonemap_extents.csv')

## Create CloneMaps

In [7]:
def create_clonemap(lonmin, latmin, lonmax, latmax, buffer=None):
    
    forcing_resolution=0.5
    
    """Add buffer to clonemap when specified."""
    if add_buffer is not None:
        coords = add_buffer(lonmin, lonmax, latmin, latmax, buffer)
        
        lonmin = coords[0]
        lonmax = coords[1]
        latmin = coords[2]
        latmax = coords[3]
        
    """Create new clonemap compatible with forcing data resolution."""
    dlon = lonmax - lonmin
    dlat = latmax - latmin

    msg = (
        "The clonemap extent divided by the forcing resolution must yield"
        "an integer number of grid cells."
    )
    assert dlon % forcing_resolution == 0, f"Longitudes not compatible. {msg}"
    assert dlat % forcing_resolution == 0, f"Latitudes not compatible. {msg}"
    
    print(lonmin, latmax, lonmax, latmin)
    
    clonemap_dir = f'{MODELDIR}/global_05min/cloneMaps'
    globalclonemap = clonemap_dir + "/clone_global_05min.map"
    outputclonemap = f"{OUTPUTDIR}/{index.lower()}_05min.map"
    print(outputclonemap)

    subprocess.call(
        f"gdal_translate -of PCRaster {globalclonemap} -projwin "
        f"{lonmin} {latmax} {lonmax} {latmin} {outputclonemap}",
        shell=True,
    )
    
    return outputclonemap

## Run all sites and create output dataframe with extents

In [12]:
site_ids = []

lonsmin = []
lonsmax = []
latsmin = []
latsmax = []

for index, row in df_sites.iterrows():
    create_clonemap(row.lon_min, row.lat_min, row.lon_max, row.lat_max, 1.5)
    coords = add_buffer(row.lon_min, row.lon_max, row.lat_min, row.lat_max, 1.5)
    
    site_ids.append(index)
    lonsmin.append(coords[0])
    lonsmax.append(coords[1])
    latsmin.append(coords[2])
    latsmax.append(coords[3])

df_extents = pd.DataFrame()
df_extents['site_id'] = site_ids
df_extents['lon_min'] = lonsmin
df_extents['lon_max'] = lonsmax
df_extents['lat_min'] = latsmin
df_extents['lat_max'] = latsmax

df_extents = df_extents.set_index('site_id')
df_extents.to_csv(f'{FLUXDIR}/clonemap_extents.csv')

139.0 -35.0 142.5 -32.5
/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/global_05min/cloneMaps/SophieBEP/au-cpr_05min.map
-55.0 -17.5 -57.5 -15.0
/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/global_05min/cloneMaps/SophieBEP/br-npw_05min.map
100.0 39.5 103.5 36.0
/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/global_05min/cloneMaps/SophieBEP/cn-ha2_05min.map
-1.5 39.0 -4.0 35.5
/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/global_05min/cloneMaps/SophieBEP/es-lgs_05min.map
2.0 45.5 5.5 42.0
/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/global_05min/cloneMaps/SophieBEP/fr-pue_05min.map
-51.5 7.5 -54.0 4.0
/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/global_05min/cloneMaps/SophieBEP/gf-guy_05min.map
101.0 5.0 104.5 1.5
/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/global_05min/cloneMaps/SophieBEP/my-pso_05min.map
4.0 54.0 7.5 50.5
/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/global_05min/cloneMaps/SophieBEP/nl-loo_05min.map
29

In [13]:
df_extents

,lon_min,lon_max,lat_min,lat_max
site_id,,,,
AU-Cpr,139.0,142.5,-32.5,-35.0
BR-Npw,-55.0,-57.5,-15.0,-17.5
CN-Ha2,100.0,103.5,36.0,39.5
ES-LgS,-1.5,-4.0,35.5,39.0
FR-Pue,2.0,5.5,42.0,45.5
GF-Guy,-51.5,-54.0,4.0,7.5
MY-PSO,101.0,104.5,1.5,5.0
NL-Loo,4.0,7.5,50.5,54.0
SD-Dem,29.0,32.5,12.0,15.5
